<h1>Wordcount with Rheem <div style="float:right; z-index:1"><img src="rheem.png" width="100px" /></div></h1>

This notebook demonstrates how to run Wordcount, the _Hello world!_ for data processing tools. To run this notebook, you will need the [Jupyter Scala kernel](https://github.com/alexarchambault/jupyter-scala).

Let us briefly give the basic idea of WordCount in the Java API:

```java
RheemContext rheemCtx = new RheemContext(new Configuration())
    .withPlugin(Spark.basicPlugin())
    .withPlugin(Java.basicPlugin());
    
JavaPlanBuilder planBuilder = new JavaPlanBuilder(rheemCtx);
Collection<Tuple2<String, Integer> wordCounts = planBuilder
    .readTextFile("hdfs://my-namenode/my-corpus.txt")
    .flatMap(line -> Arrays.asList(line.split("\\W+")))
    .map(word -> new Tuple2<>(word.toLowerCase(), 1))
    .reduceByKey(Tuple2::getField0,
                 (t1, t2) -> new Tuple2<>(t1.getField0(), t1.getField1() + t2.getField1()))
    .collect();

```

At first, we obtain an input dataset.

In [1]:
locally {
    import java.io._
    import scala.io.Source
    
    val file = new File("data/iliad.txt")
    if (!file.exists) {
        file.getParentFile.mkdirs()
        val source = Source.fromURL("http://www.gutenberg.org/cache/epub/6130/pg6130.txt")
        val writer = new BufferedWriter(new OutputStreamWriter(new FileOutputStream(file), "UTF-8"))
        source.foreach(char => writer.write(char.asInstanceOf[Int]))
        writer.close()
        source.close()
    }
}

Next, we intialize Rheem.

In [2]:
// Disable logging.
import $ivy.`org.slf4j:slf4j-nop:1.7.12`
org.slf4j.LoggerFactory.getLogger("root").info("Enforcing slf4j-nop...")

import $ivy.$                           


In [ ]:
// Load dependencies into the kernel.
import $ivy.`org.qcri.rheem::rheem-api:0.3.0`,
    $ivy.`org.qcri.rheem:rheem-basic:0.3.0`,
    $ivy.`org.qcri.rheem:rheem-java:0.3.0`,
    $ivy.`org.qcri.rheem::rheem-spark:0.3.0`,
    $ivy.`org.apache.spark::spark-core:1.6.0`,
    $ivy.`org.apache.spark::spark-graphx:1.6.0`,
    $ivy.`de.hpi.isg:profiledb-store:0.1.1`,
    $ivy.`com.github.sekruse::spark-summit-demo:1.0-SNAPSHOT`

// Do the relevant imports.
import org.qcri.rheem.api._
import org.qcri.rheem.core.api._
import org.qcri.rheem.core.optimizer.ProbabilisticDoubleInterval
import org.qcri.rheem.java.Java, org.qcri.rheem.spark.Spark
import de.hpi.isg.profiledb.store.model._
import com.github.sekruse.spark_summit_demo._

// Set up a Rheem context.
val localDir = new java.io.File(".").getAbsoluteFile
val config = new Configuration(s"file://$localDir/rheem.properties")

If this notebook is run in an offline environment, run the `run-webserver.sh` script to provide the required JS libraries.

In [ ]:
val offline = true
if (offline) {
    addModule("plotly", "http://localhost:8888/files/js/plotly-latest.min")
    addModule("d3", "http://localhost:8888/files/js/d3.v4.min")
    config.setProperty("spark.driver.host", "localhost")
}

Now, we can run the Wordcount.

In [ ]:
// Define a class to handle word counts neatly.
case class WC(word: String, count: Int) {
    def +(that: WC) = {
        require(this.word == that.word)
        WC(this.word, this.count + that.count)
    }

    override def toString: String = s"${count}x ${word}"
}

In [ ]:
locally {
    val experiment = new Experiment("my-exp", new Subject("WordCount", "1.0"))
    val rheemCtx = new RheemContext(config)
        .withPlugin(Java.basicPlugin)
        .withPlugin(Spark.basicPlugin)
    
    // Set up a new plan.
    val planBuilder = new PlanBuilder(rheemCtx)
        .withJobName("WordCount")
        .withUdfJarsOf(this.getClass)
        .withExperiment(experiment)
    
    val wordCounts = planBuilder

        // Read the text file.
        .readTextFile(s"file://$localDir/data/iliad.txt").withName("Load file")

        // Split each line by non-word characters.
        .flatMap(_.split("\\W+"), udfLoad = (in: Long, out: Long) => 100 * in).withName("Split words")
        .withTargetPlatforms(Spark.platform)

        // Filter empty tokens.
        .filter(_.nonEmpty, selectivity = 0.99).withName("Filter empty words")

        // Attach counter to each word.
        .map(word => WC(word.toLowerCase, 1)).withName("To lower case, add counter")

        // Sum up counters for every word.
        .reduceByKey(_.word, _ + _).withName("Add counters")
        .withCardinalityEstimator((in: Long) => math.round(in * 0.01))
        //.withTargetPlatforms(Spark.platform)
    
        // Mask rather small words counts.
        .map(wc => if (wc.count > 1000) wc else WC("(other)", wc.count)).withName("Mask rather small words")
        .reduceByKey(_.word, _ + _).withName("Add counters again")

        // Execute the plan and collect the results.
        .collect()
    
    publish.html("<h1>Words in Homer's Iliad</h1>")
    plotPieChart[WC](
        name = "Words in Homer's Iliad",
        data = wordCounts,
        values = _.count.toDouble,
        labels = _.word,
        showlegend = false
    )
 
    publish.html("<h1>Execution plan</h1>")
    plotExecutionPlan(experiment)
}